<a href="https://colab.research.google.com/github/guhdalla/IdentificarPassaporte/blob/main/IdentificarPassaporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyngrok

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install -q streamlit_ace

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!pip install fuzzywuzzy[speedup]

In [ ]:
from pyngrok import ngrok

In [ ]:
%%writefile app.py

from fuzzywuzzy import process
import streamlit as st
import re
import json
import requests

def main():

  def getPaises():
    response = requests.get('https://guhdalla.github.io/paises.json')
    return response.json()['PAISES']
      
  def getUrlByPais(paisField):
    paises = getPaises();
    for pais in paises:
      if pais["PAIS"].upper() == paisField.upper():
        return pais["URL"]

  def containPassaporteInUrl(url, sigla, passaporte):
    response = requests.get(url)
    json = response.json()[sigla.lower()]
    for i in json:
      if int(i[sigla.lower()]) == int(passaporte):
        return True
      else:
        return False

  def containNameInInterpol(nome):
    response = requests.get("https://cspinheiro.github.io/interpol.json")
    json = response.json()["interpol"]
    nomes = []
    for i in json:
      nomes.append(i["interpol"])

    result = process.extract(nome, nomes)
    print(result)
    for text in result: 
        if text[1] > 86:
          return True
        else:
          return False
    

  def service(passaporte, nome):
    pais = re.sub("[0-9]{9}", "",passaporte)
    passaporte = re.sub("[a-zA-Z]{3}", "",passaporte)

    url = getUrlByPais(pais);
    if url is None:
      return st.error("Pais inexistente!")
    isPassaporte = containPassaporteInUrl(url, pais, passaporte)
    isWanted = containNameInInterpol(nome)
    print("Pass", isPassaporte)
    print("Wanted", isWanted)
    if isPassaporte and not isWanted:
      st.success("Permitido.")
    else:
      if isWanted:
        st.error("Pessoa sujeito a investigação!")
      elif not isPassaporte:
        st.error("Passaporte inexistente!")
      else:
        st.error("Nao permitido!")


  def verifyNome(nome):
    if nome is "":
      return False
    match = re.search("[0-9]", nome)
    if not match:
      return True
    else:
      return False

  def verifyPassaporte(passaporte):
    match = re.search("([a-zA-Z]{3})([0-9]{9})", passaporte)
    if match:
      return True
    else:
      return False

  with st.form("form_migracao"):
    st.write("MIGRACAO")
    passaporte = st.text_input('Passaporte')
    nome = st.text_input('Nome')

    submitted = st.form_submit_button("VERIFICAR")
    if submitted:
      if verifyPassaporte(passaporte) and verifyNome(nome):
        service(passaporte, nome)
      else:
        st.write("Passaporte ou nome foram digitados incorretamente! Tente Novamente.")

if __name__=='__main__': 
    main()  

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-04-28 17:34:23.561 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.45s
your url is: https://rotten-quail-57.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.150.15:8501

[('Pedro da Silva ', 60), ('Luzia dos Anjos ', 60), ('Ricardo da Silva Leão', 60), ('Bruna de Villas Boas ', 60), ('Pietra da Silva Rebouças', 60)]
Pass True
Wanted False
[('Bruno de Oliveira Ribeiro', 86), ('Bruna de Villas Boas ', 45), ('Keila Anjos ', 43), ('Pedro da Silva ', 40), ('Pedro da Silva ', 40)]
Pass True
Wanted False
[('Bruno de Oliveira Ribeiro', 86), ('Fernando de Oliveira Ribeiro', 86), ('Marlene de Oliveira Ribeiro', 86), ('João de Oliveira Ribeiro', 86), ('Marianna de Oliveira Ribeiro', 86)]
Pass True
Wanted False
[('Bruno de Oliveira Ribeiro', 86), ('Ricardo da Silva Leão', 53), ('Pedro dos Anjos ', 50), ('Fernando dos Santos Pereira', 49), ('Pedro dos Santos Pereira', 49)]
Pass Tru